__Note__: This is a notebook to acquire and transform the raw Amazon sales rank source data into a useable dataframe. The raw data has not been included in the github repo because it is too large (7.82 GB). As a result, the code below has not been amended to pull from a non-local source. The dataframe generated by parsing and aggregating the raw data has been saved as a csv in this notebook. 

The raw data is available here: 
https://www.kaggle.com/ucffool/amazon-sales-rank-data-for-print-and-kindle-books#ranks.zip

In order to reproduce code below, please follow instructions:
- normpath - can be replaced by the extracted "ranks_norm folder" in kaggle source data
- outpath - can be replaced by 'https://github.com/jitsen-design/Data-607-Final-Project/tree/master/'
- mainpath and meta - can be replaced by the "amazon_com_extras.csv" in kaggle source data  

In [2]:
import pandas as pd
import zipfile 
import json
import os

In [ ]:
# extract "ranks norm" folder
 
#thezipfile = '/Users/euniceok/PycharmProjects/cuny/spring2019/Final Project/amazon-sales-rank-data-for-print-and-kindle-books/ranks_norm.zip'
#targetdir = '/Users/euniceok/PycharmProjects/cuny/spring2019/Final Project/amazon-sales-rank-data-for-print-and-kindle-books/'

#with zipfile.ZipFile(thezipfile,'r') as zip_ref: zip_ref.extractall(targetdir)

In [ ]:
# extract "ranks" folder

#seczipfile = '/Users/euniceok/PycharmProjects/cuny/spring2019/Final Project/amazon-sales-rank-data-for-print-and-kindle-books/ranks.zip'
#with zipfile.ZipFile(seczipfile,'r') as zip_ref: zip_ref.extractall(targetdir)

In [5]:
# parse json for just one book file sample
normpath = '/Users/euniceok/PycharmProjects/cuny/spring2019/Final Project/amazon_data/ranks_norm/'
filename = '000721393X_com_norm.json'
#filename = 'B00WPFOVS0_com_norm.json'

In [21]:
# open up the json file
with open(normpath+filename) as f:
    d = json.load(f)
    print(next(iter(d.items()))) # print first key: value pair just to check

('1501005600', 956170)


In [ ]:
d != []

In [5]:
# list out all files in the normed rankings folder
filenames = []
for it in os.listdir(normpath):
    if it.endswith(".json"):
        filenames.append(it)

In [57]:
# see how many files (books) there are
len(filenames)

66760

In [41]:
# filenames

In [18]:
normpath+filenames[:1][0]

'/Users/euniceok/PycharmProjects/cuny/spring2019/Final Project/amazon-sales-rank-data-for-print-and-kindle-books/ranks_norm/B00BFSMXB0_com_norm.json'

In [49]:
new_dict = {}

In [50]:
%%timeit

for file in filenames[:10]: 
    with open(normpath+file) as f:
        file = file.split("_")[0]
        d = json.load(f)
        if d == []:
            continue
        else:
            val_max = max(d.values())
            val_min = min(d.values())
            new_dict.update({file: (val_max, val_min)})        

20.2 ms ± 809 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [47]:
new_dict = {}

In [48]:
%%timeit

for file in filenames[:100]: 
    with open(normpath+file) as f:
        file = file.split("_")[0]
        d = json.load(f)
        if d == []:
            continue
        else:
            val_max = max(d.values())
            val_min = min(d.values())
            new_dict.update({file: (val_max, val_min)})  

199 ms ± 10.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [61]:
new_dict = {}

In [62]:
%%timeit

for file in filenames: 
    with open(normpath+file, errors='replace') as f:
        file = file.split("_")[0]
        d = json.load(f)
        if d == []:
            continue
        else:
            val_max = max(d.values())
            val_min = min(d.values())
            new_dict.update({file: (val_max, val_min)})  

3min 43s ± 4.93 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [63]:
# convert from dict to dataframe 
asrdf = pd.DataFrame.from_dict(new_dict,orient='index')
asrdf = asrdf.rename(columns={0: "max_rank", 1:"min_rank"}) 
asrdf.head()

,max_rank,min_rank
B00BFSMXB0,2699752,227614
B07CKYQ1MV,39185,1067
B06XV6XP2G,355931,35857
1609454839,1872441,39881
178133188X,823407,14391


In [65]:
asrdf.shape

(66753, 2)

In [64]:
outpath = '/Users/euniceok/PycharmProjects/cuny/spring2019/Final Project/amazon-sales-rank-data-for-print-and-kindle-books/output'
# asrdf.to_csv(outpath + '/min_maxranks.csv')

In [85]:
# create df with isbn, min sales rank, max sales rank, metadata

# pull in min/max ranks
amdf = pd.read_csv(outpath + '/min_maxranks.csv').rename(columns={'Unnamed: 0':'ASIN'})
print(amdf.head(5))

# pull in metadata
mainpath = '/Users/euniceok/PycharmProjects/cuny/spring2019/Final Project/amazon-sales-rank-data-for-print-and-kindle-books'
meta = '/amazon_com_extras.csv'
metadf = pd.read_csv(mainpath + meta, encoding='ISO-8859-1').drop(columns='Unnamed: 6')
print(metadf.head(5))

         ASIN  max_rank  min_rank
0  B00BFSMXB0   2699752    227614
1  B07CKYQ1MV     39185      1067
2  B06XV6XP2G    355931     35857
3  1609454839   1872441     39881
4  178133188X    823407     14391
         ASIN GROUP     FORMAT  \
0  1250150183  book  hardcover   
1   778319997  book  hardcover   
2  1608322564  book  hardcover   
3   310325331  book  hardcover   
4   312616295  book  hardcover   

                                               TITLE  \
0  The Swamp: Washington's Murky Pool of Corrupti...   
1            Rise and Shine, Benedict Stone: A Novel   
2  Sell or Be Sold: How to Get Your Way in Busine...   
3  Christian Apologetics: An Anthology of Primary...   
4  Gravity: How the Weakest Force in the Universe...   

                               AUTHOR                   PUBLISHER  
0                        Eric Bolling          St. Martin's Press  
1                     Phaedra Patrick              Park Row Books  
2                       Grant Cardone  Greenleaf B

In [86]:
amdf.shape

(66753, 3)

In [87]:
metadf.shape

(63748, 6)

In [89]:
master = pd.merge(amdf, metadf, on='ASIN', how='inner')

In [97]:
master = master.sort_values('min_rank', ascending=True).reset_index(drop=True)

In [99]:
master.to_csv(outpath + '/masterrankings.csv')

In [100]:
master.shape

(51898, 8)